# Data preparation
### Load packages

In [ ]:
import pandas as pd
import calendar
import os

from datetime import datetime
from ta import add_all_ta_features

### Load data

In [ ]:
Directory = 'C:/.../TFT_for_Stock_Movement_Prediction/data'

# DAX constituents
constituents_data = pd.read_csv(os.path.join(Directory, 'constituents_data.csv'), index_col = [0], header = [0, 1])

# DAX constituents for technical indicators
constituents_data_ta = pd.read_csv(os.path.join(Directory, 'constituents_data_ta.csv'), index_col = [0], header = [0, 1])

### Preparations
#### Create matrix index lists

In [ ]:
def create_index(dataset):
    date_index, stock_index = [], []
    date_index = dataset.index
    stock_index = dataset.columns.levels[1]
    return date_index, stock_index

date_index, stock_index = create_index(constituents_data)

#### Split dataset into single features

In [ ]:
def split_dataset(dataset):
    Open, High, Low, Close, Volume = [], [], [], [], []
    Open, High, Low, Close, Volume = dataset['Open'], dataset['High'], dataset['Low'], dataset['Close'], dataset['Volume']
    return Open, High, Low, Close, Volume

Open, High, Low, Close, Volume = split_dataset(constituents_data)

#### Create portfolio table

In [ ]:
# Portfolio data and ICB 
portfolio_table = pd.DataFrame(index = range(len(stock_index)))
portfolio_table['Symbol'] = stock_index
portfolio_table['Industry'] = ['Consumer Discretionary', 'Industrials', 'Financials', 'Basic Materials', 'Health Care',
                               'Consumer Staples', 'Consumer Discretionary', 'Basic Materials', 'Financials', 
                               'Consumer Discretionary', 'Financials', 'Financials', 'Industrials', 'Telecommunications',
                               'Utilities', 'Health Care', 'Basic Materials', 'Consumer Discretionary', 'Financials',
                               'Technology', 'Consumer Discretionary', 'Health Care', 'Industrials', 'Financials',
                               'Consumer Discretionary', 'Health Care', 'Industrials', 'Utilities', 'Technology',
                               'Industrials', 'Health Care', 'Basic Materials', 'Consumer Discretionary']
portfolio_table['Supersector'] = ['Consumer Products and Services', 'Industrial Goods and Services', 'Insurance', 
                                  'Chemicals', 'Health Care', 'Personal Care, Drug and Grocery Stores', 
                                  'Automobiles and Parts', 'Chemicals', 'Banks', 'Automobiles and Parts', 
                                  'Financial Services', 'Banks', 'Industrial Goods and Services', 'Telecommunications',
                                  'Utilities', 'Health Care', 'Basic Resources', 'Consumer Products and Services',
                                  'Insurance', 'Technology', 'Automobiles and Parts', 'Health Care', 
                                  'Industrial Goods and Services', 'Insurance', 'Automobiles and Parts', 'Health Care',
                                  'Industrial Goods and Services', 'Utilities', 'Technology', 
                                  'Industrial Goods and Services', 'Health Care', 'Chemicals', 'Automobiles and Parts']

### Feature engineering 
#### Target and return feature

In [ ]:
# Close to Close Return
CCR = pd.DataFrame(index = range(len(date_index)), columns = stock_index)
for i in range(len(date_index)):
    for j in range(len(stock_index)):
        CCR.iloc[i, j] = ((Close.iloc[i, j] / Close.iloc[i - 1, j]) - 1)
CCR = CCR.iloc[1 : , : ].reset_index(drop = True)

#### Time features

In [ ]:
# Date
Date = pd.DataFrame(date_index)

# Weekday
Weekday = pd.DataFrame(index = range(len(date_index)), columns = ['Weekday'])
for i in range(len(date_index)):
    Weekday['Weekday'][i] = datetime.strptime(Date['Date'][i], '%Y-%m-%d').weekday()
Weekday['Weekday'] = Weekday['Weekday'].astype('category')
Weekday = Weekday.iloc[1 : , : ].reset_index(drop = True)

# Week
Week = pd.DataFrame(index = range(len(date_index)), columns = ['Week'])
for i in range(len(date_index)):
    Week['Week'][i] = datetime.strptime(Date['Date'][i], '%Y-%m-%d').isocalendar().week
Week['Week'] = Week['Week'].astype('category')
Week['Week'] = Week['Week'].replace(53, 52)
Week = Week.iloc[1 : , : ].reset_index(drop = True)

# Month
Month = pd.DataFrame(index = range(len(date_index)), columns = ['Month'])
for i in range(len(date_index)):
    Month['Month'][i] = datetime.strptime(Date['Date'][i], '%Y-%m-%d').month
Month['Month'] = Month['Month'].astype('category')
Month = Month.iloc[1 : , : ].reset_index(drop = True)

# Day of month
Monthday = pd.DataFrame(index = range(len(date_index)), columns = ['Monthday'])
Monthday['Monthday'] = pd.to_datetime(Date.Date, format = '%Y-%m-%d').dt.day.astype('category')
Monthday = Monthday.iloc[1 : , : ].reset_index(drop = True)

# Select features
time_features = ['Weekday', 'Week', 'Month', 'Monthday']

#### Basic historical features

In [ ]:
## Interday categorical changes
def create_interday(price):
    dataset = pd.DataFrame(index = range(len(date_index)), columns = stock_index)
    for i in range(len(date_index)):
        for j in range(len(stock_index)):
            if (price.iloc[i, j] >= price.iloc[i - 1, j]):
                dataset.iloc[i, j] = 1
            else:
                dataset.iloc[i, j] = 0
    dataset = dataset.iloc[1 : , : ].reset_index(drop = True)
    return dataset

COO = create_interday(Open) # Open to Open
CHH = create_interday(High) # High to High
CLL = create_interday(Low) # Low to Low
CCC = create_interday(Close) # Close to Close

## Intraday continuous changes
def create_intraday(price1, price2):
    dataset = pd.DataFrame(index = range(len(date_index)), columns = stock_index)
    for i in range(len(date_index)):
        for j in range(len(stock_index)):
            dataset.iloc[i, j] = (price1.iloc[i, j] - price2.iloc[i, j])
    dataset = dataset.iloc[1 : , : ].reset_index(drop = True)
    return dataset

LHR = create_intraday(High, Low) # Low to High range
OCR = create_intraday(Close, Open) # Open to Close range

# High range
HR = pd.DataFrame(index = range(len(date_index)), columns = stock_index)
for i in range(len(date_index)):
    for j in range(len(stock_index)):
        if (Close.iloc[i, j] >= Open.iloc[i, j]):
            HR.iloc[i, j] = (High.iloc[i, j] - Close.iloc[i, j])
        else:
            HR.iloc[i, j] = (High.iloc[i, j] - Open.iloc[i, j])
HR = HR.iloc[1 : , : ].reset_index(drop = True)

# Low range
LR = pd.DataFrame(index = range(len(date_index)), columns = stock_index)
for i in range(len(date_index)):
    for j in range(len(stock_index)):
        if (Close.iloc[i, j] >= Open.iloc[i, j]):
            LR.iloc[i, j] = (Open.iloc[i, j] - Low.iloc[i, j])
        else:
            LR.iloc[i, j] = (Close.iloc[i, j] - Low.iloc[i, j])
LR = LR.iloc[1 : , : ].reset_index(drop = True)

## Select features
bh_categorical_features = ['COO', 'CHH', 'CLL', 'CCC']
bh_continuous_features = ['LHR', 'OCR', 'HR', 'LR']
bh_features = bh_categorical_features + bh_continuous_features

#### Technical indicatiors

In [ ]:
# Extract prices
Open, High, Low, Close, Volume = split_dataset(constituents_data_ta)

# Get indicator names
temp = pd.DataFrame()
temp['Open'], temp['High'], temp['Low'], temp['Close'], temp['Volume'] = Open[stock_index[0]], High[stock_index[0]], Low[stock_index[0]], Close[stock_index[0]], Volume[stock_index[0]]
temp_df = add_all_ta_features(temp, open = 'Open', high = 'High', low = 'Low', close = 'Close', volume = 'Volume', fillna = True)
ta_names = temp_df.columns[5 : ]

# Create indicatior dataframes
for i in ta_names:
     exec('{} = pd.DataFrame(columns = stock_index)'.format(i))
        
# Fill dataframes
Open, High, Low, Close, Volume = split_dataset(constituents_data_ta)
for i in range(len(stock_index)):
    temp = pd.DataFrame()
    temp['Open'], temp['High'], temp['Low'], temp['Close'], temp['Volume'] = Open[stock_index[i]], High[stock_index[i]], Low[stock_index[i]], Close[stock_index[i]], Volume[stock_index[i]]
    temp_df = add_all_ta_features(temp, open = 'Open', high = 'High', low = 'Low', close = 'Close', volume = 'Volume', fillna = True)
    temp_df = temp_df.iloc[43 : , : ].reset_index(drop = True)
    for j in range(len(ta_names)):
        locals()[ta_names[j]][stock_index[i]] = temp_df[ta_names[j]]
        
# Select features
indicator_features = ['volume_adi', 'volume_obv', 'volatility_atr', 'volatility_bbh', 'volatility_bbl', 'trend_macd', 'trend_sma_fast', 'trend_ema_fast', 'trend_adx', 'momentum_rsi', 'momentum_stoch', 'momentum_roc']

### Save data

In [ ]:
## Target and return feature
CCR.to_csv('data/CCR.csv')

## Features
# Time features
for i in range(len(time_features)):
    locals()[time_features[i]].to_csv(f'data/{time_features[i]}.csv') 
pd.DataFrame(time_features).to_csv(f'data/time_features.csv')

# Basic historical features
for i in range(len(bh_features)):
    locals()[bh_features[i]].to_csv(f'data/{bh_features[i]}.csv')
pd.DataFrame(bh_categorical_features).to_csv(f'data/bh_categorical_features.csv')
pd.DataFrame(bh_continuous_features).to_csv(f'data/bh_continuous_features.csv')
pd.DataFrame(bh_features).to_csv(f'data/bh_features.csv')

# Technical indicators
for i in range(len(indicator_features)):
    locals()[indicator_features[i]].to_csv(f'data/{indicator_features[i]}.csv')
pd.DataFrame(indicator_features).to_csv(f'data/indicator_features.csv')

## Portfolio table
portfolio_table.to_csv('data/portfolio_table.csv')